<a href="https://colab.research.google.com/github/arfild/dw_matrix_car/blob/master/day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model (model, feats):
  X = df[feats].values
  y = df['price_value'].values


  # model = DecisionTreeRegressor(max_depth=5)
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)


run_model(DecisionTreeRegressor(max_depth=5), cat_feats )

(-19695.13091100928, 148.72570644015792)

##DecisionTree


In [0]:
run_model(DecisionTreeRegressor(max_depth=5),cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest


In [0]:
model = RandomForestRegressor(max_depth=5,n_estimators=50,random_state=0)
run_model(model,cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators':50,
    'learning_rate': 0.1, 
    'seed': 0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats )

[21:58:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:58:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:58:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [0]:
m = xgb.XGBRegressor(**xgb_params)
m.fit (X,y,)

imp = PermutationImportance(m, random_state=2020).fit(X,y)
eli5.show_weights(m, feature_names=cat_feats)

[22:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.0951,param_faktura-vat__cat
0.0949,feature_kamera-cofania__cat
0.0669,feature_łopatki-zmiany-biegów__cat
0.0664,param_napęd__cat
0.0560,param_skrzynia-biegów__cat
0.0423,feature_asystent-pasa-ruchu__cat
0.0411,param_stan__cat
0.0329,feature_światła-led__cat
0.0279,feature_bluetooth__cat
0.0263,feature_regulowane-zawieszenie__cat


In [0]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc__cat', 
  'param_rok-produkcji__cat', 
  'param_pojemność-skokowa__cat', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

In [0]:
model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[22:27:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14242.8316835859, 148.71839978475074)

In [0]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:

df['param_rok-produkcji'] = df['param_rok-produkcji']. map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [0]:
eats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc__cat', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa__cat', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[22:29:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14242.8316835859, 148.71839978475074)

In [0]:

df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [0]:
df['param_moc'] = df['param_moc']. map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0])) #    ||-1 if str(x) == 'None' else int(x))
df['param_moc'].unique()

array([ 90, 115, 262, 110, 310, 105, 140, 175, 125, 185, 190, 440, 141,
       200, 224,  75,  99, 184, 109, 233, 116,  68, 286, 126, 160, 135,
       120, 272,  -1, 150, 180, 136, 102, 131, 218, 245, 170, 112, 250,
       252,  73, 100, 313, 101, 285,  70, 383, 174, 277, 132, 130, 215,
        60, 330, 163, 177,  98,  78, 189, 156, 143,  69, 113,  65, 122,
        82, 251,  95, 197, 235, 238, 171, 381, 400, 178,  80, 165,  85,
       258, 142, 204, 124,  55, 144, 231, 248, 152, 181, 210, 340, 129,
       147,  50,  54, 290, 306, 193,  77, 164,  96, 194, 111, 166, 206,
       118, 360, 211, 271, 455, 280, 106, 114, 421,  74, 213, 121, 275,
       435, 384, 326,  88, 220, 260,  64,  86, 128, 256, 240, 244, 162,
       237, 350,  35, 265, 202, 133,  83, 117, 146,  92, 192, 145, 525,
       254, 182, 328, 367, 148, 456,  97, 270, 107, 108, 203, 155,  94,
        93, 241,  20,  71, 173,  58, 205, 236,   1, 557,  84, 457,  72,
       295, 134, 425, 228,  81, 230, 201,  87, 234, 299, 585, 20

In [0]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa__cat', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )

[22:33:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:33:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:33:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10668.21096354992, 65.60418228047648)

In [0]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [0]:
feats = [
  'feature_kamera-cofania__cat', 
  'feature_łopatki-zmiany-biegów__cat', 
  'param_napęd__cat', 
  'param_skrzynia-biegów__cat', 
  'param_faktura-vat__cat', 
  'feature_asystent-pasa-ruchu__cat', 
  'param_stan__cat', 
  'feature_światła-led__cat', 
  'feature_bluetooth__cat',
  'feature_regulowane-zawieszenie__cat', 
  'feature_wspomaganie-kierownicy__cat', 
  'feature_system-start-stop__cat', 
  'feature_światła-do-jazdy-dziennej__cat', 
  'feature_światła-xenonowe__cat', 
  'feature_czujniki-parkowania-przednie__cat', 
  'param_moc', 
  'param_rok-produkcji', 
  'param_pojemność-skokowa', 
  'feature_asystent-parkowania__cat', 
  'seller_name__cat']

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats )


[22:35:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:35:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10418.18729276653, 72.80569459652894)